# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import seaborn as sns

In [4]:
# loading the data:
data = pd.read_csv('tic-tac-toe.csv')

data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [5]:
data = pd.get_dummies(data)
data.head()

,class,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,...,MR_x,BL_b,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x
0,True,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,1,0,0,1,0
1,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
2,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,1
3,True,0,0,1,0,0,1,0,0,1,...,0,0,1,0,1,0,0,1,0,0
4,True,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0


In [30]:
X = data.drop('class', axis=1)
y = data['class']

In [31]:
#the data is alrady normalized

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [43]:
import tensorflow.keras as keras

import tensorflow as tf

print(tf.__version__)

2.3.0


In [44]:
#error below: Please provide data which shares the same first dimension.

X_train.shape, X_test.shape

((766, 27), (192, 27))

In [46]:
len(X_train)

766

In [55]:
#A sequential model is what you're going to use most of the time. 
#It just means things are going to go in direct order. A feed forward model. 
model = tf.keras.models.Sequential()

#Now, we'll pop in layers. 
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))

#Let's add another identical layer for good measure.
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))

#Now, we're ready for an output layer:
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

#Now we need to "compile" the model. 
#This is where we pass the settings for actually optimizing/training the model we've defined.

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3)

Epoch 1/3
24/24 [==============================] - 0s 1ms/step - loss: 1.4184 - accuracy: 0.5679
Epoch 2/3
24/24 [==============================] - 0s 1ms/step - loss: 0.5829 - accuracy: 0.7206
Epoch 3/3
24/24 [==============================] - 0s 3ms/step - loss: 0.4980 - accuracy: 0.7376


In [56]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print("Loss:", round(val_loss,2)*100, "%")
print("Accuracy:", round(val_acc,2)*100, "%")

6/6 [==============================] - 0s 1ms/step - loss: 0.4850 - accuracy: 0.7500
Loss: 49.0 %
Accuracy: 75.0 %


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [54]:
y_pred = model.predict(X_test[:10])
np.argmax(y_pred, axis=1),y_test[:10]
# got 3 out of the 10 wrong

(array([1, 1, 1, 1, 0, 0, 1, 1, 1, 0]), 280     True
 422     True
 200     True
 613     True
 630    False
 753    False
 505     True
 208     True
 119     True
 803    False
 Name: class, dtype: bool)

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [62]:
model = tf.keras.models.Sequential()

#Now, we'll pop in layers. 
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))

#Let's add another identical layer for good measure.
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))

#Now, we're ready for an output layer:
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

#Now we need to "compile" the model. 
#This is where we pass the settings for actually optimizing/training the model we've defined.

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)

val_loss, val_acc = model.evaluate(X_test, y_test)
print("Loss:", round(val_loss,2)*100, "%")
print("Accuracy:", round(val_acc,2)*100, "%")

Epoch 1/10
24/24 [==============================] - 0s 1ms/step - loss: 1.3258 - accuracy: 0.5875
Epoch 2/10
24/24 [==============================] - 0s 1ms/step - loss: 0.5684 - accuracy: 0.7141
Epoch 3/10
24/24 [==============================] - 0s 1ms/step - loss: 0.5060 - accuracy: 0.7428
Epoch 4/10
24/24 [==============================] - 0s 1ms/step - loss: 0.4599 - accuracy: 0.7585
Epoch 5/10
24/24 [==============================] - 0s 2ms/step - loss: 0.4206 - accuracy: 0.8003
Epoch 6/10
24/24 [==============================] - 0s 2ms/step - loss: 0.3621 - accuracy: 0.8551
Epoch 7/10
24/24 [==============================] - 0s 2ms/step - loss: 0.3030 - accuracy: 0.9008
Epoch 8/10
24/24 [==============================] - 0s 1ms/step - loss: 0.2697 - accuracy: 0.9151
Epoch 9/10
24/24 [==============================] - 0s 1ms/step - loss: 0.2074 - accuracy: 0.9517
Epoch 10/10
6/6 [==============================] - 0s 1ms/step - loss: 0.1526 - accuracy: 0.9688
Loss: 15.0 %
Accuracy

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
#10